In [1]:
import wandb
import ultralytics
from ultralytics import YOLO
import torch
import cv2
import pandas as pd
import torch
import os

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True" #to help with memory allocation issues(weak gpu)
torch.cuda.empty_cache() #weak gpu
#check yolo version and using cuda
# print(ultralytics.__version__, torch.cuda.is_available())

In [3]:
!nvidia-smi

Thu Feb  6 23:36:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:06:00.0  On |                  N/A |
| 32%   52C    P0             29W /  125W |     764MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
wandb.login()

wandb: Currently logged in as: ejento47 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Loading the model yoloV8 

In [5]:
#training model
model = YOLO('yolov8n.pt')


Arguments setting for the model

In [6]:
args = dict(
    data = "data.yaml",
    epochs = 10,
    patience = 5,
    batch = 4,
    save = True,
    save_period = 5,
    device = '0',
    workers = 4,
    project = "weights",
    name = "try",
    exist_ok = True, #overwrite the existing run for iterative testing, turn false when confident
    pretrained = True,
    optimizer = 'auto',
    resume = False,
    amp = False, #mixed precision training for faster training, might cause some issues
    fraction = 0.1, #fraction of dataset if resouce limited, change to 1 when confident
    profile = False, #enable ONNX and TensorRT profiling
    freeze = None, #freeze the first N layers or can specify the layers by index
    #for validation part
    # conf = 0.01,
    # iou = 0.6,
    # degrees = 0.0, 
    # shear = 0.0,
    # perspective = 0.0,
    plots = True,
    half = True, #use half precision for weaker gpu
)

In [7]:
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.3.72 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.9.21 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=5, batch=4, imgsz=640, save=True, save_period=5, cache=False, device=0, workers=4, project=weights, name=try, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=0.1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

c:\Users\johna\anaconda3\envs\fyp\lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning D:\FYP Datasets\fyp\training\train\labels.cache... 4740 images, 296 backgrounds, 0 corrupt: 100%|██████████| 4740/4740 [00:00<?, ?it/s]Scanning D:\FYP Datasets\fyp\training\train\labels.cache... 4740 images, 296 backgrounds, 0 corrupt: 100%|██████████| 4740/4740 [00:00<?, ?it/s]
val: Scanning D:\FYP Datasets\fyp\training\val\labels.cache... 10665 images, 696 backgrounds, 0 corrupt: 100%|██████████| 10665/10665 [00:00<?, ?it/s]Scanning D:\FYP Datasets\fyp\training\val\labels.cache... 10665 images, 696 backgrounds, 0 corrupt: 100%|██████████| 10665/10665 [00:00<?, ?it/s]


Plotting labels to weights\try\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to weights\try
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all      10665       9969      0.877      0.851       0.86      0.664

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      1.09G     0.8631     0.5418     0.9574          4        640: 100%|██████████| 1185/1185 [01:48<00:00, 10.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:19<00:00, 16.76it/s]


                   all      10665       9969      0.874      0.857      0.863      0.665

10 epochs completed in 0.541 hours.
Optimizer stripped from weights\try\weights\last.pt, 6.2MB
Optimizer stripped from weights\try\weights\best.pt, 6.2MB

Validating weights\try\weights\best.pt...
Ultralytics 8.3.62  Python-3.9.21 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
Model summary (fused): 168 layers, 3,006,818 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:09<00:00, 19.13it/s]


                   all      10665       9969      0.873      0.858      0.863      0.665
               Glottis       9969       9969      0.873      0.858      0.863      0.665
Speed: 0.3ms preprocess, 2.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to weights\try
